In [14]:
# coding=utf-8
import json
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from nltk.translate.bleu_score import corpus_bleu,sentence_bleu
from rouge import Rouge
from sentence_transformers import SentenceTransformer, util
sys.setrecursionlimit(2000)

class DatasizeError(Exception):
    def __init__(self, message) :
        super().__init__(message)
        self.message=message

class SampleError(Exception):
    def __init__(self, message) :
        super().__init__(message)
        self.message=message

class CaseidError(Exception):
    def __init__(self, message) :
        super().__init__(message)
        self.message=message

error_msg={
    1: "Wrong data size",
    2: "Wrong sample format",
    3: "Wrong case id"
}

def dump_2_json(info, path):
    with open(path, 'w') as output_json_file:
        json.dump(info, output_json_file)

def report_error_msg(detail, showMsg, out_p):
    error_dict=dict()
    error_dict['errorDetail']=detail
    error_dict['errorMsg']=showMsg
    error_dict['score']=0
    error_dict['scoreJson']={}
    error_dict['success']=False
    dump_2_json(error_dict,out_p)

def report_score(score, out_p):
    result = dict()
    result['success']=True
    total_score = score['Edit_acc']['final_score'] * 0.2 + score['portability']['final_score'] * 0.35 + score['locality']['final_score']  * 0.35 + score['fluency'] * 0.1
    result['score'] = total_score
    result['scoreJson'] = {'score': total_score, 'Edit_acc':score['Edit_acc']['final_score'], 'portability':score['portability']['final_score'], 'locality':score['locality']['final_score'], 'fluency':score['fluency']}
    dump_2_json(result,out_p)

def sample_format(sample_list):
    tag=True
    for x in sample_list:                                                          
        list1 = x.keys()
        # list2 = x['pre'].keys()
        list3 = x['requested_rewrite'].keys()
        list4 = x['post'].keys()
        # if(list(list1)!=['pre', 'case_id', 'requested_rewrite', 'post']):
        if(list(list1)!=['case_id', 'requested_rewrite', 'post']):
            tag=False
            break
        # elif(list(list2)!=['rewrite_ans','rephrase_ans','portability_ans'] and list(list2)!=['rewrite_ans','rephrase_ans','locality_ans','portability_ans']):
        #     tag=False
        #     break
        elif('target_new' not in list3 or 'portability' not in list3 or 'locality' not in list3):
            tag=False
            break
        # fluency_new
        elif(list(list4)!=['rewrite_ans','rephrase_ans','locality_ans','portability_ans','fluency', 'fluency_new'] and\
              list(list4)!=['rewrite_ans','rephrase_ans','portability_ans','fluency', 'fluency_new']):
            tag=False
            break  
    return tag

def test_case_id(sample_list):
    tag =True
    for x in range(len(sample_list)-1):
        if(sample_list[x+1]['case_id']!=sample_list[x]['case_id']+1):
            tag = False
            break
    return tag

def check_format(submit_p):
    with open(submit_p, 'r',encoding='utf-8') as file:
        if 'log' in submit_p:
            lines = file.readlines()
            submit_file = []
            for line in lines:
                submit_file.append(json.loads(line))
        else:
            submit_file=json.load(file)
    # if len(submit_file)!=700:
    #     raise DatasizeError("Wrong data size")
    if (not sample_format(submit_file)):
        raise SampleError("Wrong sample format")
    if (not test_case_id(submit_file)):
        raise CaseidError("Wrong case id")

def compute_acc(answers,outputs):
    # model_path = './paraphrase-multilingual-MiniLM-L12-v2'
    model_path = '/share/huggingface/paraphrase-multilingual-MiniLM-L12-v2'
    bleu_scores = []
    rouge1s=[]
    rouge2s=[]
    rougels=[]
    rouge = Rouge()
    for an,ou in zip(answers,outputs):
        score = sentence_bleu([an], ou)
        bleu_scores.append(score)
        scores = rouge.get_scores(ou,an)
        rouge1s.append(scores[0]['rouge-1']['r'])
        rouge2s.append(scores[0]['rouge-2']['r'])
        rougels.append(scores[0]['rouge-l']['r'])

    temp_metrics = {}
    temp_metrics['BLEU SCORE'] = sum(bleu_scores) / len(bleu_scores)
    temp_metrics['ROUGE-1'] = sum(rouge1s) / len(rouge1s)
    temp_metrics['ROUGE-2'] = sum(rouge2s) / len(rouge2s)
    temp_metrics['ROUGE-L'] = sum(rougels) / len(rougels)

    model = SentenceTransformer(model_path, device="cpu")

    embeddings1 = model.encode(answers, convert_to_tensor=True)
    embeddings2 = model.encode(outputs, convert_to_tensor=True)

    # Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    temp_metrics['Bert Score'] = cosine_scores.diagonal().mean().item()
    temp_metrics['final_score'] = (temp_metrics['ROUGE-L']+temp_metrics['Bert Score'])/2
    temp_metrics['final_score'] = temp_metrics['final_score']*100
    
    return temp_metrics

def eval_score(result_path):
    with open(result_path, 'r', encoding='utf-8') as file:
        if 'log' in result_path:
            lines = file.readlines()
            print(f'data size: {len(lines)}')
            data = []
            for line in lines:
                data.append(json.loads(line))
        else:
            data=json.load(file)
    metrics = {}

    #evaluate Edit_acc
    rewrite_answer = [i['requested_rewrite']['target_new'] for i in data]
    rewrite_outputs = [i['post']['rewrite_ans'] for i in data]
    metrics['Edit_acc'] = compute_acc(rewrite_answer,rewrite_outputs)

    #evaluate portability
    portability_answer = []
    portability_outputs = []
    for item in data:
        for an in item['requested_rewrite']['portability']['por_hop']['ground_truth']:
            portability_answer.append(an)
        for ou in item['post']['portability_ans']:
            portability_outputs.append(ou)
    metrics['portability'] = compute_acc(portability_answer,portability_outputs)

    #evaluate locality
    locality_answer = []
    locality_outputs = []
    for item in data:
        if ('locality_ans' not in item['post'].keys() or len(item['requested_rewrite']['locality']['loc_hop']['prompt'])==0):
            continue
        for an in item['requested_rewrite']['locality']['loc_hop']['ground_truth']:
            locality_answer.append(an)
        for ou in item['post']['locality_ans']:
            locality_outputs.append(ou)
    if len(locality_answer)!=0 and len(locality_outputs)!=0:
        metrics['locality'] = compute_acc(locality_answer,locality_outputs)
    else:
        metrics['locality'] = {'final_score': 0}

     #evaluate fluency
    fluencys = [i['post']['fluency']['ngram_entropy'] for i in data]
    metrics['fluency'] = sum(fluencys) / len(fluencys) *10

    return metrics

In [16]:
root_dir = '../ccks2024_output/type2_80/result'

import os
file_names = os.listdir(root_dir)
print(file_names)

for file in file_names:
    submit_path = os.path.join(root_dir, file)
    out_path = os.path.join('./metrics_results', file)
    check_format(submit_path)
    score = eval_score(submit_path)
    report_score(score, out_path)
    print(file)
    print(score)


['type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p99.05_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json', 'type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p99.95_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json', 'type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-max-bs20_120_p95.0_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json', 'type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-max-bs20_120_p90.0_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json', 'type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p95.0_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json', 'type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p90.0_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json']


/home/lyc/miniconda3/envs/ke2torch23cu121/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/lyc/miniconda3/envs/ke2torch23cu121/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/lyc/miniconda3/envs/ke2torch23cu121/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order i

type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p99.05_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json
{'Edit_acc': {'BLEU SCORE': 0.19888794244459945, 'ROUGE-1': 0.0, 'ROUGE-2': 0.0, 'ROUGE-L': 0.0, 'Bert Score': 0.6562885046005249, 'final_score': 32.814425230026245}, 'portability': {'BLEU SCORE': 0.027854831558574544, 'ROUGE-1': 0.0, 'ROUGE-2': 0.0, 'ROUGE-L': 0.0, 'Bert Score': 0.511116087436676, 'final_score': 25.5558043718338}, 'locality': {'final_score': 0}, 'fluency': 39.349758044645455}
type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p99.95_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json
{'Edit_acc': {'BLEU SCORE': 0.034384220649134795, 'ROUGE-1': 0.0, 'ROUGE-2': 0.0, 'ROUGE-L': 0.0, 'Bert Score': 0.5138594508171082, 'final_score': 25.692972540855408}, 'portability': {'BLEU SCORE': 0.02485757812355424, 'ROUGE-1': 0.0, 'ROUGE-2': 0.0, 'ROUGE-L': 0.0, 'Bert Score': 0.48513516783714294, 'final_score': 24.256758391857147}, 'locality': {'f

单个

In [ ]:
submit_path = '../ccks2024_output/type2_80/type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p90.0_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json'
out_path = './metrics_results/type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p90.0_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json'

try:
    check_format(submit_path)
    score = eval_score(submit_path)
    report_score(score, out_path)
except DatasizeError as e:
    print(e)
    check_code = 1
    report_error_msg(error_msg[check_code],error_msg[check_code], out_path)
except SampleError as e:
    print(e)
    check_code = 2
    report_error_msg(error_msg[check_code],error_msg[check_code], out_path)
except CaseidError as e:
    print(e)
    check_code = 3
    report_error_msg(error_msg[check_code],error_msg[check_code], out_path)


In [ ]:
submit_path = '../ccks2024_output/type2_80/type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p95.0_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json'
out_path = './metrics_results/type2_80-Qwen-1_8B-Chat-CKnowEdit-0-24-mlp.c_proj-knb_dict-mean-bs20_120_p95.0_rsTrue_a1_pd0_bias_knb_only_t_loss0.001.json'

try:
    check_format(submit_path)
    score = eval_score(submit_path)
    report_score(score, out_path)
except DatasizeError as e:
    print(e)
    check_code = 1
    report_error_msg(error_msg[check_code],error_msg[check_code], out_path)
except SampleError as e:
    print(e)
    check_code = 2
    report_error_msg(error_msg[check_code],error_msg[check_code], out_path)
except CaseidError as e:
    print(e)
    check_code = 3
    report_error_msg(error_msg[check_code],error_msg[check_code], out_path)
